## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from collections import Counter

%matplotlib inline

In [2]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [3]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [4]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self,
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            raise ValueError('invalid criterion name')

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            raise ValueError('invalid max_features name')

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # 1. Мера неопределенности родительского узла
        i_p = self.__gini_p((l_c + r_c) / (l_s + r_s))
        # 2. Мера неопределенности левого дочернего узла
        i_l = self.__gini_p(l_c / l_s)
        # 3. Мера неопределенности правого дочернего узла
        i_r = self.__gini_p(r_c / r_s)
        return self.__impurity(i_p, i_l, i_r, l_s, r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # 1. Мера неопределенности родительского узла
        i_p = self.__entropy_p((l_c + r_c) / (l_s + r_s))
        # 2. Мера неопределенности левого дочернего узла
        i_l = self.__entropy_p(l_c / l_s)
        # 3. Мера неопределенности правого дочернего узла
        i_r = self.__entropy_p(r_c / r_s)
        return self.__impurity(i_p, i_l, i_r, l_s, r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # 1. Мера неопределенности родительского узла
        i_p = self.__misclass_p((l_c + r_c) / (l_s + r_s))
        # 2. Мера неопределенности левого дочернего узла
        i_l = self.__misclass_p(l_c / l_s)
        # 3. Мера неопределенности правого дочернего узла
        i_r = self.__misclass_p(r_c / r_s)
        return self.__impurity(i_p, i_l, i_r, l_s, r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.sqrt(n_feature)))
        return feature_ids[:max_features]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.log2(n_feature)))
        return feature_ids[:max_features]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем значения свойства по возрастанию и приводим к аналогичной последовательности целевые переменные.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Количество меток класса.
        class_number = np.unique(y).shape[0]

        # Здесь, по всей видимости, мы обрезаем образцы с учетом параметра min_samples_split, для того, 
        # чтобы в середине осталось необходимое количество образцов размера min_samples_split. 
        # Хотя это немного странный подход, можно проверять min_samples_split заранее.
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # Сравниваем каждый элемент с предыдущим, таким образом мы получаем информацию о том насколько неоднородная выборка,
        # это будет иметь влияния на вычисление меры неоднородности.
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Если выборка однородная, то нет необходимости делать расщепление.
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что происходит деталях объяснить не так легко, но судя по отладочной информации, 
        # основной смысл данного участка кода - сформировать различные варианты разбиения классов, 
        # на основе которых можно вычислить различные меры неоднородности и выбрать из них наиболее оптимальную.
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] \
            + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)

        # Здесь создаются различные варианты разбиения классов по узлам дерева:
        # l_class_count - варианты разбиения для левого узла
        # r_class_count - варианты разбиения для правого узла
        # l_sizes - общее число образцов в левом узле
        # r_sizes - общее число образцов в правом узле
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Вычисляем возможные меры неоднородности для всех вариантов разбиения классов.
        # Выбираем индекс наименьшей из них, как наиболее оптимальный. 
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # По индексу лучшей неоднородности, определяем количество элементов в левом узле.
        # Возвращаем значение лучшей меры неоднородности и среднее значение элемента выборки 
        # (сам элемент и его сосед слева), который и будет являться тем самым параметром threshold, 
        # по которому будет определяться разделение выборки на правый и левый узлы.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Если узел содержит только образцы одного класса, то останавливаем расщепление.
        if np.unique(y).shape[0] == 1:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                y[0],
                0,
                0,
                y,
                'Node contains only samples of one class',
            )
            return

        # Если доля образцов одного класса больше или равна необходимой доли для расщепления, то останавливаем расщепление.
        most_common_y = Counter(y).most_common(1).pop()
        if most_common_y[1] >= self.sufficient_share * y.shape[0]:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                most_common_y[0],
                0,
                0,
                y,
                'Samples of one class is greater than or equal sufficient_share',
            )
            return

        # Если достигнута максимальная глубина дерева, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if self.max_depth == depth:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                most_common_y[0],
                0,
                0,
                y,
                'Maximum depth of the tree',
            )
            return

        # Если количество образцов меньше требуемого для разделения узла, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if y.shape[0] < self.min_samples_split:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                most_common_y[0],
                0,
                0,
                y,
                'Number of samples is less than min_samples_split',
            )
            return
        
        n_samples, n_features = x.shape
        split_data = []
        # Ищем признак по которому будем делать разбиение (который ведет к самому большому приросту информации).
        # Для этого необходимо вычислить для каждого признака меру неоднородности (impurity).
        # Чем ниже неоднородность, тем выше прирост информации.
        # Также здесь мы вычисляем порог (threshold), 
        # по которому будем определять идти в правый узел дерева или левый (т.к. дерево у нас бинарное).
        for feature_id in self.get_feature_ids(n_features):
            impurity, threshold = self.__find_threshold(x[:,feature_id], y)
            if threshold is not None:
                split_data.append((impurity, threshold, feature_id,))

        # Если недостаточно данных для разбиения, например, 
        # достигнут предел минимального количества образцов, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if not split_data:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                most_common_y[0],
                0,
                0,
                y,
                'Empty split data',
            )
            return

        best_split = min(split_data, key=lambda x: x[0])
            
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, best_split[2], best_split[1])

        # Если после расщепления, какой либо узел оказался пустой, то значит, данный узел расщепить невозможно, 
        # поэтому он будет являться листом.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if y_left.shape[0] == 0 or y_right.shape[0] == 0:
            self.tree[node_id] = (
                self.LEAF_TYPE,
                most_common_y[0],
                0,
                0,
                y,
                'After splitting one of the nodes is empty',
            )
            return
        else:
            self.tree[node_id] = (self.NON_LEAF_TYPE, best_split[2], best_split[1], best_split[0], y, None)
            self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, *_ = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, *_ = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    @staticmethod
    def __gini_p(p):
        return 1 - (p ** 2).sum(axis=1)

    @staticmethod
    def __entropy_p(p):
        return - np.nansum(p * np.log2(p), axis=1)

    @staticmethod
    def __misclass_p(p):   
        return 1 - p.max(axis=1)

    @staticmethod
    def __impurity(i_p, i_l, i_r, l_s, r_s):
        t_s = l_s + r_s
        return i_p - (np.squeeze(np.asarray(l_s / t_s)) * i_l) - (np.squeeze(np.asarray(r_s / t_s)) * i_r)


In [5]:
df_debug = df.sample(n=100)
x_debug = df_debug.as_matrix(columns=df_debug.columns[1:])
y_debug = df_debug.as_matrix(columns=df_debug.columns[:1])
y_debug = y_debug.reshape(y_debug.shape[0])

In [6]:
my_clf_debug = MyDecisionTreeClassifier(min_samples_split=2)
my_clf_debug.fit(x_debug, y_debug)

In [7]:
def print_tree(tree, node_id=0, depth=0, df=None, df_shift=0):
    node = tree.get(node_id)
    if node:
        tab = '\t' * depth
        node_type, feature, threshold, impurity, y, message = node
        y_cnt = Counter(y)
        node_type = 'LEAF' if node_type == 1 else 'NON_LEAF'
        if df is not None:
            feature = df.columns[feature + df_shift]
        if node_type == 'NON_LEAF':
            print('\n'.join('{}{}: {}'.format(tab, k, v) for k, v in {
                'type': node_type,
                'depth': depth,
                'threshold': threshold,
                'feature': feature,
                'impurity': impurity,
                'samples': '[{}]'.format(', '.join('{}:{}'.format(k, v) for k, v in y_cnt.items())),
            }.items()))
        else:
            print('\n'.join('{}{}: {}'.format(tab, k, v) for k, v in {
                'type': node_type,
                'depth': depth,
                'class': threshold,
                'samples': '[{}]'.format(', '.join('{}:{}'.format(k, v) for k, v in y_cnt.items())),
                'message': message,
            }.items()))
        print('{}------------------------------'.format(tab))
        print_tree(tree, 2 * node_id + 1, depth + 1, df)
        print_tree(tree, 2 * node_id + 2, depth + 1, df)

In [8]:
print_tree(my_clf_debug.tree, df=df_debug, df_shift=1)

type: NON_LEAF
depth: 0
threshold: 5000.0
feature: MonthlyIncome
impurity: 5.128205128247798e-06
samples: [0:96, 1:4]
------------------------------
	type: LEAF
	depth: 1
	class: 0
	samples: [0:49, 1:1]
	message: After splitting one of the nodes is empty
	------------------------------
	type: NON_LEAF
	depth: 1
	threshold: 1.0
	feature: NumberOfTimes90DaysLate
	impurity: 5.8823529412374564e-06
	samples: [0:47, 1:3]
	------------------------------
		type: LEAF
		depth: 2
		class: 0
		samples: [0:4]
		message: Node contains only samples of one class
		------------------------------
		type: NON_LEAF
		depth: 2
		threshold: 46.5
		feature: RevolvingUtilizationOfUnsecuredLines
		impurity: 7.8764965343453e-06
		samples: [0:43, 1:3]
		------------------------------
			type: NON_LEAF
			depth: 3
			threshold: 62.5
			feature: RevolvingUtilizationOfUnsecuredLines
			impurity: 0.00016088486676724134
			samples: [0:28, 1:2]
			------------------------------
				type: NON_LEAF
				depth: 4
				thr

In [9]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [10]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print('MyDecisionTreeClassifier:', t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print('DecisionTreeClassifier:', t2 - t1)

MyDecisionTreeClassifier: 2.271366834640503
DecisionTreeClassifier: 1.233086109161377


## Проверка качества работы

In [11]:
gkf = KFold(n_splits=5, shuffle=True)

In [12]:
print('MyDecisionTreeClassifier accuracy_score:')
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    # Изначально было y_pred=clf.predict(X_test)
    # Но мне кажется, этот вариант не совсем корректный
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

MyDecisionTreeClassifier accuracy_score:
0.928909952607
0.929325683878
0.928743660098
0.933275130955
0.931734087224


In [13]:
print('DecisionTreeClassifier accuracy_score:')
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

DecisionTreeClassifier accuracy_score:
0.890537956265
0.892658185749
0.893780660181
0.89307391702
0.891655926496


# Применить для задачи Titanic 

In [14]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [15]:
df_train = df_train.fillna(df_train.median(axis=0), axis=0)
df_train['SexCode'] = df_train['Sex'].map(lambda sex: 1 if sex == 'male' else 0)
df_train['AgeGroup'] = df_train['Age']
df_train['AgeGroup'] = df_train['AgeGroup'].map(lambda age: int(age // 10) + 1)

In [16]:
df_train2 = df_train.drop(([
    'PassengerId',
    'Name', 
    'Sex', 
    'Ticket',
    'Cabin',
    'Embarked',
]), axis=1)

df_train2.head(5)

,Survived,Pclass,Age,SibSp,Parch,Fare,SexCode,AgeGroup
0,0,3,22.0,1,0,7.2500,1,3
1,1,1,38.0,1,0,71.2833,0,4
2,1,3,26.0,0,0,7.9250,0,3
3,1,1,35.0,1,0,53.1000,0,4
4,0,3,35.0,0,0,8.0500,1,4


In [17]:
x_train = df_train2.as_matrix(columns=df_train2.columns[1:])
y_train = df_train2.as_matrix(columns=df_train2.columns[:1])
y_train = y_train.reshape(y_train.shape[0])

print(x_train)
print(y_train)

[[  3.      22.       1.     ...,   7.25     1.       3.    ]
 [  1.      38.       1.     ...,  71.2833   0.       4.    ]
 [  3.      26.       0.     ...,   7.925    0.       3.    ]
 ..., 
 [  3.      28.       1.     ...,  23.45     0.       3.    ]
 [  1.      26.       0.     ...,  30.       1.       3.    ]
 [  3.      32.       0.     ...,   7.75     1.       4.    ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

In [20]:
best_score = float('-inf')
best_min_samples_split = None
best_max_depth = None
best_sufficient_share = None
for i in range(10):
    for j in map(lambda x: x * 2, range(4)):
        if j == 0:
            j = None
        for k in map(lambda x: x / 10, range(4, 12, 2)):
            my_clf = MyDecisionTreeClassifier(min_samples_split=i, max_depth=j, sufficient_share=k)
            my_clf.fit(x_train2, y_train2)
            a_s = accuracy_score(y_pred=my_clf.predict(x_test2), y_true=y_test2)
            if a_s > best_score:
                best_score = a_s
                best_min_samples_split = i
                best_max_depth = j
                best_sufficient_share = k
print('best_score', best_score)
print('best_min_samples_split', best_min_samples_split)
print('best_max_depth', best_max_depth)
print('best_sufficient_share', best_sufficient_share)

best_score 0.723880597015
best_min_samples_split 1
best_max_depth None
best_sufficient_share 1.0


Как видно, на тренировочной выборке наилучшая прогноза 0.723880597015. Посмотрим, что получится на тестовой выборке kaggle:

In [21]:
df_test = df_test.fillna(df_test.median(axis=0), axis=0)
df_test['SexCode'] = df_test['Sex'].map(lambda sex: 1 if sex == 'male' else 0)
df_test['AgeGroup'] = df_test['Age']
df_test['AgeGroup'] = df_test['AgeGroup'].map(lambda age: int(age // 10) + 1)

In [22]:
df_test2 = df_test.drop(([
    'PassengerId',
    'Name', 
    'Sex', 
    'Ticket',
    'Cabin',
    'Embarked',
]), axis=1)
df_test2.head(5)

,Pclass,Age,SibSp,Parch,Fare,SexCode,AgeGroup
0,3,34.5,0,0,7.8292,1,4
1,3,47.0,1,0,7.0000,0,5
2,2,62.0,0,0,9.6875,1,7
3,3,27.0,0,0,8.6625,1,3
4,3,22.0,1,1,12.2875,0,3


In [29]:
x_test = df_test2.as_matrix()
print(x_test)

[[  3.      34.5      0.     ...,   7.8292   1.       4.    ]
 [  3.      47.       1.     ...,   7.       0.       5.    ]
 [  2.      62.       0.     ...,   9.6875   1.       7.    ]
 ..., 
 [  3.      38.5      0.     ...,   7.25     1.       4.    ]
 [  3.      27.       0.     ...,   8.05     1.       3.    ]
 [  3.      27.       1.     ...,  22.3583   1.       3.    ]]


In [30]:
my_clf = MyDecisionTreeClassifier(min_samples_split=1)
my_clf.fit(x_train, y_train)

In [31]:
print_tree(my_clf.tree, df=df_test2)

type: NON_LEAF
depth: 0
threshold: 28.0
feature: Age
impurity: 4.1241486892218404e-09
samples: [0:549, 1:342]
------------------------------
	type: NON_LEAF
	depth: 1
	threshold: 4.0
	feature: AgeGroup
	impurity: 2.328636894421976e-09
	samples: [1:142, 0:210]
	------------------------------
		type: NON_LEAF
		depth: 2
		threshold: 1.0
		feature: Pclass
		impurity: 1.280198933528709e-07
		samples: [0:102, 1:61]
		------------------------------
			type: NON_LEAF
			depth: 3
			threshold: 1.0
			feature: SibSp
			impurity: 3.3159643107738557e-07
			samples: [1:19, 0:63]
			------------------------------
				type: LEAF
				depth: 4
				class: 0
				samples: [0:1]
				message: Node contains only samples of one class
				------------------------------
				type: NON_LEAF
				depth: 4
				threshold: 54.0
				feature: Age
				impurity: 2.8017608516561765e-07
				samples: [1:19, 0:62]
				------------------------------
					type: LEAF
					depth: 5
					class: 0
					samples: [1:3, 0:12]
					me

In [32]:
y_pred=my_clf.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0,

In [33]:
# sample_submission_tree.csv
df_predicted = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_pred})
df_predicted.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [34]:
df_predicted.to_csv('sample_submission_tree.csv', sep=',', index=False)

В итоге лучший результат на kaggle - 'находится в обработке...'. Что хуже логистической регрессии и kNN. Возможно, просто алгоритм не слишком подходит к этой задаче или дерево нуждается в доработке.